[View in Colaboratory](https://colab.research.google.com/github/david7joy/RNNProjects/blob/master/Generate_Dinasour_Names.ipynb)

In [2]:
ls /content/drive/ColabNotebooks/ProjectRNN/

Baby Name Generator.ipynb  babynames.txt  dinos.txt  logs/  setting drive.ipynb


In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
tf.reset_default_graph()

In [0]:
import numpy as np
from tensorflow.contrib import rnn
import random
import os
import time

In [0]:
logs_path = '/content/drive/ColabNotebooks/ProjectRNN/logs/'
writer = tf.summary.FileWriter(logs_path)

In [126]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
os.chdir('/content/drive/ColabNotebooks/ProjectRNN/')

In [157]:
file = os.listdir()
file = file[2]
file

'dinos.txt'

In [0]:
with open(file) as f:
    training_data = f.read()
    training_data = training_data.replace('\n',' ')
    training_data = training_data.lower()

In [0]:
chars = list(set(training_data))

In [160]:
data_size,vocab_size = len(training_data),len(chars)
print("The size of the data is %d and vocab size %d"%(data_size,vocab_size))

The size of the data is 19909 and vocab size 27


In [0]:
char_to_ix = {ch:i for i,ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i,ch in enumerate(sorted(chars))}

In [0]:
# no of hidden cells or units in rnn
n_hidden = 256

# hyperparameters 
learning_rate = .001
training_iters = 50000
display_step = 2000
time_steps = 7
num_input = 1
batch_size = 512

In [0]:
x = tf.placeholder("float",[batch_size,time_steps,num_input])  # the size of each input is 1*19
y = tf.placeholder("float",[batch_size,vocab_size])

In [0]:
weights = {'out':tf.Variable(tf.random_normal([n_hidden,vocab_size]))}
biases = {'out':tf.Variable(tf.random_normal([vocab_size]))}

In [0]:
def RNN(x,weights,biases):
    x = tf.reshape(x,[-1,time_steps])
    x = tf.split(x,time_steps,1)
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    rnn_cell = rnn.DropoutWrapper(rnn_cell, output_keep_prob=0.8)
    #rnn_cell = rnn.BasicLSTMCell(n_hidden)
    outputs,states = rnn.static_rnn(rnn_cell,x,dtype=tf.float32)
    check1,check2 = outputs,states
    return tf.matmul(outputs[-1],weights['out']) + biases['out']

In [0]:
pred = RNN(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
grads_and_vars = optimizer.compute_gradients(cost)
capped_gvs = [(tf.clip_by_value(grad, -5, 5), var) for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

In [0]:
# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
#initializing the variable

init = tf.global_variables_initializer()

start_time = time.time()
with tf.Session() as sess:
    sess.run(init)
    total_loss = 0
    total_acc  = 0
    offset = 0
    r1 = offset
    r2 = offset+(batch_size*time_steps)
    
    #saver = tf.train.Saver()
    for steps in range(1, training_iters + 1):
        if offset > 16200:
            offset = 0
            r1=offset
            r2 = offset+(batch_size*time_steps)
            

        #getting the input sequence
        #print("steps :%d offset1: %d range: %d" %(steps,offset,batch_size*time_steps))
        #print(r1,r2,steps)
        X = [char_to_ix[str(training_data[i])] for i in range(r1, r2)]
        X = np.reshape(np.array(X),[batch_size,time_steps,1])
        batch_size = X.shape[0]
        r1 = r2
        r2 = r2+(batch_size*time_steps)

        # getting ready the True Y
        zeros_array = np.zeros((batch_size,vocab_size), dtype=float)
        for i in range(batch_size):
            zeros_array[i][char_to_ix[str(training_data[offset + time_steps])]] = 1.0
            offset += time_steps
            
        Y = np.reshape(zeros_array, [batch_size, -1])
        
        #_, acc, cost,one_hot_pred = sess.run([optimizer, accuracy, cost, pred], feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        _, acc, loss, onehot_pred = sess.run([training_op, accuracy, cost, pred], feed_dict={x: X, y: Y})
        total_loss += loss
        total_acc += acc
        if steps % display_step == 0:
            print("Iter= " + str(steps+1) + ", Average Loss= " + "{:.6f}".format(total_loss/display_step) + ", Average Accuracy= " + "{:.2f}%".format((100*total_acc)/display_step))
            total_loss = 0
            total_acc = 0


    # Save the variables to disk.
    #save_path = saver.save(sess, "/Users/davidjoy/Desktop/Github:Study Material/6341 - Applied Machine Learning/Rnn/Dinosaur Project")
    #print("Model saved in path: %s" % save_path)
    
    #sampling
    print("Total Mins {}".format((time.time()-start_time)/60))
    while True:
      inputwords = "enter 3 letters " 
      enteredletters = input(inputwords)
      letters = enteredletters.strip()
      letters = list(letters)
      if len(letters) != time_steps:
            continue
      try:
        letters = [char_to_ix[i] for i in letters]
        for i in range(50):
          fixing_shape = np.zeros((batch_size,time_steps,1), dtype=float)
          cell_input = np.reshape(np.array(letters), [-1, time_steps, 1])
          cell_input = cell_input+fixing_shape
          pred_letter = sess.run(pred,feed_dict={x:cell_input})
          pred_letter = np.reshape(np.array(pred_letter[1]),(1,vocab_size))
          pred_location = int(tf.argmax(pred_letter,axis=1).eval())
          pred_location
          enteredletters += "%s"%(ix_to_char[pred_location])
          letters.append(pred_location)
          letters = letters[1:]
          name = enteredletters.split()[0]
#         for i in names:
        print(name)
      except:
        print("letter not in dictionary")

Iter= 2001, Average Loss= 1.132593, Average Accuracy= 65.18%
Iter= 4001, Average Loss= 0.266838, Average Accuracy= 90.09%
Iter= 6001, Average Loss= 0.236318, Average Accuracy= 90.96%
Iter= 8001, Average Loss= 0.225620, Average Accuracy= 91.24%
Iter= 10001, Average Loss= 0.219362, Average Accuracy= 91.40%
Iter= 12001, Average Loss= 0.215926, Average Accuracy= 91.50%
Iter= 14001, Average Loss= 0.213870, Average Accuracy= 91.57%
Iter= 16001, Average Loss= 0.212035, Average Accuracy= 91.58%
Iter= 18001, Average Loss= 0.210928, Average Accuracy= 91.62%
Iter= 20001, Average Loss= 0.209886, Average Accuracy= 91.66%
Iter= 22001, Average Loss= 0.209285, Average Accuracy= 91.67%
Iter= 24001, Average Loss= 0.208734, Average Accuracy= 91.68%
Iter= 26001, Average Loss= 0.208092, Average Accuracy= 91.68%
Iter= 28001, Average Loss= 0.207792, Average Accuracy= 91.70%
Iter= 30001, Average Loss= 0.207294, Average Accuracy= 91.72%
Iter= 32001, Average Loss= 0.207011, Average Accuracy= 91.73%
Iter= 34001,